In [1]:
import torch
import torch.nn as nn
import os
import numpy as np
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, utils, models
from torch.nn import functional as F
from torch.autograd import Variable
from tensorboardX import SummaryWriter
from tqdm import trange, tqdm
from PIL import Image
import torch.utils.model_zoo as model_zoo
import pandas as pd
import bcolz
import pickle

In [2]:
root_dir = 'tiny-imagenet-200/train/'
writer = SummaryWriter()
batch_size = 16
np.random.seed(42)
num_classes = 5

In [3]:
with open('tiny-imagenet-200/words.txt') as file:
    a = file.readlines()
a = [i[:-1].split('\t') for i in a]
clss = [i[0] for i in a]
a = [i[1].replace(',', '').split() for i in a]

In [4]:
words = []
idx = 0
word2idx = {}
vectors = bcolz.carray(np.zeros(1), rootdir=f'glove.6B/6B.50.dat', mode='w')

with open(f'glove.6B/glove.6B.50d.txt', 'rb') as f:
    for l in f:
        line = l.decode().split()
        word = line[0]
        words.append(word)
        word2idx[word] = idx
        idx += 1
        vect = np.array(line[1:]).astype(np.float)
        vectors.append(vect)
    
vectors = bcolz.carray(vectors[1:].reshape((400001, 50)), rootdir=f'glove.6B/6B.50.dat', mode='w')
vectors.flush()
pickle.dump(words, open(f'glove.6B/6B.50_words.pkl', 'wb'))
pickle.dump(word2idx, open(f'glove.6B/6B.50_idx.pkl', 'wb'))

In [5]:
vectors = bcolz.open(f'glove.6B/6B.50.dat')[:]
words = pickle.load(open(f'glove.6B/6B.50_words.pkl', 'rb'))
word2idx = pickle.load(open(f'glove.6B/6B.50_idx.pkl', 'rb'))

glove = {w: vectors[word2idx[w]] for w in words}

In [6]:
a = [[glove[j] for j in i if j in glove] for i in a]
a = [np.mean(i, axis = 0) for i in a]
names = {}
for i in range(len(clss)):
    names[clss[i]] = a[i]

/home/david/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/david/.local/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [7]:
classes = os.listdir(root_dir)
classes = [i for i in classes if not type(names[i]) == np.float64]
test_classes = np.random.choice(classes, size = 20, replace = False)
classes = [i for i in classes if i not in test_classes]
att = 1
def fin_dir(cl):
    return root_dir + cl + '/images/'

In [8]:
class MatchingNetwork(nn.Module):
    def __init__(self, fce = True):
        super(MatchingNetwork, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 64, 3, 1, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, 1, 1),
            nn.BatchNorm2d(128),
            nn.ReLU())
        self.conv_size = 128
        self.fce = fce
        self.size = self.conv_size * 49
        self.dist = nn.CosineSimilarity()
        self.softmax = nn.Softmax()
        self.fc = nn.Sequential(
            nn.Linear(2 * self.size + 1, 128),
            nn.Sigmoid(),
            nn.Linear(128, 1))
        self.att = nn.Linear(50, self.conv_size)

    def forward(self, support_set, inp, names):
        inp = self.cnn(inp).view(-1, self.conv_size, 49)
        def func(i):
            sup = self.cnn(support_set[:, i]).view(-1, self.conv_size, 49)
            name = self.att(names[:, i])
            att_score1 = self.softmax(torch.bmm(name.unsqueeze(1), inp))
            att_score2 = self.softmax(torch.bmm(name.unsqueeze(1), sup))
            out1 = (inp * att_score1).view(-1, self.size)
            out2 = (sup * att_score2).view(-1, self.size)
            out = self.fc(torch.cat([torch.abs(out1 - out2), (out1 - out2) ** 2, self.dist(out1, out2).view(-1, 1)], dim = 1))
            return out
        return torch.stack([func(i) for i in range(5)])

In [9]:
images = []
for cl in tqdm(classes):
    images.append([])
    for img_name in os.listdir(fin_dir(cl)):
        img = np.array(Image.open(fin_dir(cl) + img_name).resize((28, 28)), dtype = 'float32') / 255
        if not len(img.shape) == 3:
            img = np.stack([img, img, img])
        else:
            img = img.transpose((2, 0, 1))
        images[-1].append(img)

100%|██████████| 177/177 [00:33<00:00,  5.29it/s]


In [10]:
one_shot_images = []
for cl in tqdm(test_classes):
    one_shot_images.append([])
    for img_name in os.listdir(fin_dir(cl)):
        img = np.array(Image.open(fin_dir(cl) + img_name).resize((28, 28)), dtype = 'float32') / 255
        if not len(img.shape) == 3:
            img = np.stack([img, img, img])
        else:
            img = img.transpose((2, 0, 1))
        one_shot_images[-1].append(img)

100%|██████████| 20/20 [00:03<00:00,  5.49it/s]


In [11]:
class ImageDataset(Dataset):
    def __init__(self, images, classes1):
        self.images = images
        self.classes = classes1
        self.n = len(images)
        self.cur_class = 0
        self.cur_img = 0
        self.size = 0
        for i in images:
            self.size += len(i)
        
    def __len__(self):
        return self.size
        
    def __getitem__(self, idx):
        img1 = self.images[self.cur_class][self.cur_img]
        ind = np.random.randint(0, self.n, size = 5)
        if not self.cur_class in ind:
            res = np.random.randint(0, 5)
            ind[res] = self.cur_class
        else:
            res = np.where(ind == self.cur_class)[0][0]
        class_names = [self.classes[i] for i in ind]
        sup = np.stack([self.images[a][np.random.randint(0, len(self.images[a]))] for a in ind])
        self.cur_img += 1
        if self.cur_img >= len(self.images[self.cur_class]):
            self.cur_img = 0
            self.cur_class = (self.cur_class + 1) % self.n
        sample = {'img' : torch.from_numpy(img1.astype('float32')), 'res' : int(res), 'support_set': sup, 'class_names' : torch.Tensor([names[i] for i in class_names])}
        return sample

In [12]:
dataset = ImageDataset(images, classes)
dataloader = DataLoader(dataset, batch_size = batch_size)
one_shot_dataset = ImageDataset(one_shot_images, test_classes)
one_shot_dataloader = DataLoader(one_shot_dataset, batch_size = batch_size)

In [13]:
model = MatchingNetwork(False)
model.cuda()
coef = 0.0

In [14]:
sgd = torch.optim.Adam(model.parameters(), weight_decay = coef)
criterion = nn.CrossEntropyLoss()
t = trange(800)
for i in t:
    model.train()
    sum_loss = 0
    acc = 0
    for j, batch in enumerate(dataloader):
        target = Variable(batch['res']).cuda()
        img = batch['img']
        support_set = batch['support_set']
        y_pred = model.forward(Variable(support_set).cuda(), Variable(img).cuda(), Variable(batch['class_names']).cuda()).view(-1, 5)
        loss = criterion(y_pred, target)
        _, x = torch.max(y_pred, dim = 1)
        sum_loss += loss
        acc += torch.sum(x == target).float() / target.shape[0]
        t.set_description('accuracy = %g, loss = %g' % (acc / (j + 1), loss))
        sgd.zero_grad()
        loss.backward()
        sgd.step()
    writer.add_scalar('cross entropy', sum_loss / len(dataloader), i)
    writer.add_scalar('train accuracy', acc / len(dataloader), i)
    model.eval()
    one_shot_acc = 0
    for j, batch in enumerate(one_shot_dataloader):
        target = Variable(batch['res']).cuda()
        y_pred = model.forward(Variable(batch['support_set']).cuda(), Variable(batch['img']).cuda(), Variable(batch['class_names']).cuda())
        _, x = torch.max(y_pred, dim = 1)
        one_shot_acc += torch.mean((x == target).float())
    writer.add_scalar('one shot accuracy', one_shot_acc / len(one_shot_dataloader), i)

  0%|          | 0/800 [00:00<?, ?it/s]/home/david/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:32: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/david/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:33: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
accuracy = 0.198888, loss = 1.61241:   2%|▏         | 12/800 [22:38<24:46:34, 113.19s/it]

KeyboardInterrupt: 

In [15]:
torch.save(model, 'MatchingNetwork2')

/home/david/.local/lib/python3.6/site-packages/torch/serialization.py:159: UserWarning: Couldn't retrieve source code for container of type MatchingNetwork. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
